# Law, Order, and Algorithms
## A statistical approach to assessing _reasonable suspicion_

In [0]:
# Load packages
library(tidyverse)
library(ROCR)

# Set some initial options
options(digits = 3)
theme_set(theme_bw())

## Background

"Stop-and-frisk" is a police practice of temporarily detaining, questioning, and at times searching civilians on the street for weapons and other contraband.

In New York City, between 2003 and 2013, over 100,000 stops were made per year, with 685,724 people being stopped at the height of the practice in 2011. You can read more about this [here](https://en.wikipedia.org/wiki/Stop-and-frisk_in_New_York_City).

The vast majority of those stopped were African-American or Latinx, raising concerns of racial bias. Further, there's concern that many stops were conducted without "reasonable suspicion" of criminal activity, in violation of the Fourth Amendment. 

In this lab, we will work with a sample of stops in NYC made between 2007 and 2013, recorded on a [UF-250 form](https://www.prisonlegalnews.org/news/publications/blank-uf-250-form-stop-question-and-frisk-report-worksheet-nypd-2016/), to explore a statistical approach to assessing potential Fourth Amendment violations.

In [0]:
# Load data
stops <- read_rds("../data/sqf_sample.rds")

# Preprocess the data
cols_to_keep <- c('found_weapon', 'frisked', 'suspected_crime',
                  'precinct',
                  'suspect_race', 'suspect_sex', 'suspect_build',
                  'month', 'day',
                  'stopped_bc_furtive', 'stopped_bc_other', 'stopped_bc_object', 'stopped_bc_drugs', 'stopped_bc_violent', 
                  'stopped_bc_lookout', 'stopped_bc_clothing', 'stopped_bc_casing', 'stopped_bc_bulge', 'stopped_bc_desc',
                  'inside_outside', 'location_housing',
                  'additional_highcrime', 'additional_investigation', 'additional_proximity', 'additional_direction', 'additional_other',
                  'additional_associating', 'additional_sights', 'additional_time', 'additional_evasive', 'additional_report',
                  'radio_run', 'officer_uniform',
                  'suspect_height', 'suspect_weight', 'observation_period', 'suspect_age', 'local_hit_rate', 'year')
stops <- stops %>%
  select(any_of(cols_to_keep)) %>%
  # only keep stops with information in all entries
  na.omit() %>%
  mutate(
    # convert non-numerical variables into factors, grouping values occurring < 0.1% of the time into "Other"
    across(where(is.character) | where(is.logical) | where(is.factor), 
           ~ fct_lump(factor(.x), prop = 0.001)),
    # convert outcome into numeric
    across(all_of('found_weapon'), ~ if_else(.x == "TRUE", 1, 0)),
    frisked = as.logical(frisked),
    suspect_race = if_else(suspect_race == "white", "white", str_to_title(suspect_race))
  )

head(stops)

## The data

This week, we will use the same data frame as last week: `stops`, a sample of stops in NYC, recorded on a 
[UF-250 form][uf250_link].
As a reminder, below is a list of columns in the data, roughly corresponding to the [UF-250 form][uf250_link] (for simplicity, we work with a subset of the available columns):

* Base information regarding stop:
    * `id`, `serial`, `year`, `date`, `day`, `time`, `hour`, `precinct`, `location_housing`, 
      `suspected_crime`, `lat`, `lon`, `xcoord`, `ycoord`

* Circumstances which led to the stop:
    * `stopped_bc_object`, `stopped_bc_desc`, `stopped_bc_casing`, `stopped_bc_lookout`, `stopped_bc_clothing`, `stopped_bc_drugs`, `stopped_bc_furtive`, `stopped_bc_violent`, `stopped_bc_bulge`, `stopped_bc_other`
    
* Suspect demographics:
    * `suspect_dob`, `suspect_id_type`, `suspect_sex`, `suspect_race`,
      `suspect_hispanic`, `suspect_age`, `suspect_height`, `suspect_weight`,
      `suspect_hair`, `suspect_eye`, `suspect_build`, `reason_explained`,
      `others_stopped`

* Whether physical force was used:
    * `force_hands`, `force_wall`, `force_ground`, `force_drawn`,
      `force_pointed`, `force_baton`, `force_handcuffs`,
      `force_pepper`, `force_other`

* Was suspect arrested?: `arrested`
    * if yes: `arrested_reason`

* Was summons issued?: `summons_issued`

* Officer in uniform?: `officer_uniform`, `officer_verbal`, `officer_shield`

* Was person frisked?: `frisked`
    * if yes: `frisked_bc_suspected_crime`, `frisked_bc_weapons`, `frisked_bc_attire`, `frisked_bc_actual_crime`, `frisked_bc_noncompliance`, `frisked_bc_threats`, `frisked_bc_prior`, `frisked_bc_furtive`, `frisked_bc_bulge`

* Was person searched?: `searched`,
    * if yes: `searched_hardobject`, `searched_outline`,
      `searched_admission`, `searched_other`

* Was weapon found?: `found_weapon`
    * if yes: `found_gun`, `found_pistol`, `found_rifle`, `found_assault`,
      `found_knife`, `found_machinegun`, `found_other`
      
* Was other contraband found?: `found_contraband`

* Additional circumstances/factors
    * `additional_report`, `additional_investigation`, `additional_proximity`, 
      `additional_evasive`, `additional_associating`, `additional_direction`, 
      `additional_highcrime`, `additional_time`, `additional_sights`, 
      `additional_other`

* Additional reports prepared: `extra_reports`

* Whether the stop was a result of a radio run: `radio_run`

* Was the stop made inside or outside: `inside_outside`

* Period of observation (minutes): `observation_period`, `stop_length`

* Type of ID the person had (photo, refused, verified, other): `identification`

* Local hit rate: `local_hit_rate`, defined as the average hit rate for drug and cpw stops at the location of the stop for the previous year, using gaussian kernel averaging. NA if it's other type of stops.

[uf250_link]: https://www.prisonlegalnews.org/media/publications/Blank%20UF-250%20Form%20-%20Stop%2C%20Question%20and%20Frisk%20Report%20Worksheet%2C%20NYPD%2C%202016.pdf
      

## Analysis

For this lab, we will focus on stops for suspicion of criminal possession of a weapon (CPW).

#### Exercise: Data filtering

In [0]:
# Your code here!
# Create a data frame named stops_cpw with only cpw stops as indicated by column suspected_crime


One might say that a CPW stop was _ex post_ justified if a weapon was ultimately recovered. To gauge whether such a stop was _ex ante_ justified, we will use the available data to build a risk assessment model, allowing us to estimate the likelihood a stopped individual is carrying a weapon. We can then examine the model-estimated risk scores to see whether people were stopped even though the available evidence suggests that they are unlikely to be carrying a weapon, potentially in violation of the Fourth Amendment. We will look at each of these steps — estimating risk and examining scores — in turn below.

### Estimating risk

For an individual $i$, let $X_i$ be all the information we know about the
individual prior to making the stop decision, and define


$$
Y_i =
\begin{cases}
1 & \textrm{if a weapon is found on individual } i \\
0 & \textrm{otherwise}
\end{cases}
$$


Then, the _risk_ we are interested in measuring can be written as

$$
r_i = \Pr(Y_i = 1 ~|~ X_i).
$$



In other words, we wish to estimate the probability that we will recover a 
weapon from an individual, characterized by covariates $X_i$, if we were to 
stop (and frisk) them.

One way to estimate this probability is to fit a logistic regression on the
subset of individuals who were stopped, to predict whether a weapon will be
found, using all the covariates that an officer observed prior to conducting the
stop. 

To fit and evaluate our risk model, we split the data into two parts: a training set and a test set.
We train our logistic regression model using the training set and evaluate it on the test set, so that when evaluting the model, we won't get an overly optimistic measure of performance.
You can read more about training and test sets at [this page](https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets).

In [0]:
# Split train-test set
stops_train_cpw <- stops_cpw %>%
  filter(year %in% 2009:2010)

stops_test_cpw <- stops_cpw %>%
  filter(year %in% 2011:2012)

We'll start be estimating risk based on whether the officer reported seeing a "bulge" (indicative of carrying a weapon) and whether the officer indicated that the stop occurred in a high-crime neighborhood. We train and make predictions with the logistic regression model as follows:

In [0]:
# Specifying the model
features <- c('stopped_bc_bulge', 'additional_highcrime')

# We will use all covariates listed in `features` to predict the outcome "found_weapon" 
glm_formula <- reformulate(features, response="found_weapon")

# Train model
glm_fit <- glm(glm_formula, data = stops_train_cpw, family = binomial(link = "logit"))

# Make predictions on the test set
predictions <- predict(glm_fit, newdata = stops_test_cpw, type = "response")

There are many ways to evaluate a regression model's performance.
For a binary classification task like the one we are working on (predicting whether or not someone is carrying a weapon), a common performance metric is [AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). Given a random pair of people in the , exactly one of whom is carrying a weapon, the AUC is the probability that the model will correctly identify the person with the weapon. A perfect binary prediction model will thus give an AUC of 1 while a model that is making random guesses will produce an AUC of 0.5.

In [0]:
# Helper function to calculate AUC
compute_auc <- function(probs, labels) {
  pred <- prediction(probs, labels)
  auc <- performance(pred, 'auc')
  auc <- unlist(slot(auc, 'y.values'))
  auc
}

compute_auc(probs = predictions, labels = stops_test_cpw$found_weapon)

In the above, we fit a very simple model based on only two features. Now let's see if we can achieve a higher AUC by incluiding more covariates in the logistic regression model.

#### Exercise: Building a better logistic regression model

In [0]:
# Your code here!
# Build a better logistic regression model here by including more covariates

# check the AUC of the new model
compute_auc(probs = predictions, labels = stops_test_cpw$found_weapon)

#### Exercise: Interpreting risk scores

Discuss your risk model. 
Do you think it gives an accurate estimate of the likelihood someone has a weapon, 
based on information observed by the officer?

### Assessing 4A violations

We'll now use our model estimates to assess potential Fourth Amendment violations. To start, we'll compute the fraction of CPW stops that have less than 1% _ex ante_ chance of recovering a weapon.

#### Exercise: Calculate the proportion of low-risk stops

In [0]:
# Add our predictions to the data frame
stops_test_cpw <- stops_test_cpw %>%
  mutate(found_weapon_pred = predictions)

# Given a threshold p between 0 and 1 (e.g., 0.01)
p <- 0.01
# Calculate the proportion of stops with smaller than p ex-ante chance of recovering a weapon

# Your code here!


#### Exercise: Interpreting the result

Discuss the result above. Do you think this provides evidence of Fourth Amendment violations?

Above we computed the proportion of stops with less than 1% chance of turning up a weapon. Now we'll repeat this exercise for every threshold (e.g., 0.1% and 2%). 
(This is called the [cumulative distirbution function (CDF)](https://en.wikipedia.org/wiki/Cumulative_distribution_function) of the risk distribution.)

#### Exercise: Calculate the proportion of stops having less than risk $p$, for different values of $p$

In [0]:
# Your code here!
# create a data frame named risk_dist with 2 columns: p and prop_stops
# representing the proportion of stops having risk less than or equal to p,
# for all values of p.


We can now plot the proportion of stops less than each risk level. As a point of comparison, we also plot the a dashed line showing the overall hit rate in the dataset.

In [0]:
hit_rate_overall <- mean(stops_test_cpw$found_weapon)

ggplot(risk_dist, aes(x = p, y = prop_stops)) +
  geom_vline(xintercept=hit_rate_overall, linetype='dashed') +
  geom_line() +
  scale_x_continuous('Probability of weapon recovery', labels = scales::percent,
                     limits = c(0, 0.2), expand = expansion(mult = c(0, 0.01))) +
  scale_y_continuous('Proportion of stops', labels = scales::percent,
                     limits = c(0, 1), expand = expansion(mult = c(0, 0.01))) 

#### Exercise: Interpreting the result

Discuss the result above. What do you think the plot above tells us about Fourth Amendment violations?

Now let's generate a version of the plot above disaggregated by race, for those categorized in the dataset as `Black`, `Hispanic`, or `white`.

#### Exercise: Calculate the risk distribution disaggregated by race

In [0]:
# Your code here!
# The output should be a data frame named risk_dist_by_race 
# containing 3 columns: suspect_race, p, and prop_stops


#### Exercise: Hit rate by race

In [0]:
# Your code here!
# For a point of reference in the plot, compute the overall hit rate 
# for Black, Hispanic, and white individuals.
# The output should be a data frame named hit_rate_by_race with 2 columns: suspect_race and hit_rate


We now plot the results.

In [0]:
# weapon recovery probability vs. CDF
ggplot(risk_dist_by_race, aes(x = p, y = prop_stops, color = suspect_race)) +
  geom_vline(data=hit_rate_by_race, aes(xintercept=hit_rate, color = suspect_race), linetype='dashed') +
  geom_line() +
  scale_x_continuous('Probability of weapon recovery', labels = scales::percent,
                     limits = c(0, .2), expand = expansion(mult = c(0, 0.01))) +
  scale_y_continuous('CDF', labels = scales::percent,
                     limits = c(0, 1), expand = expansion(mult = c(0, 0.01))) +
  scale_color_discrete("Suspect race")


#### Exercise: Interpreting the result

Discuss the result above. What do you think the plot above tells us about Fourth Amendment violations? What does it tell us about discrimination?